In [1]:
from collections import Counter, defaultdict
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from gensim.corpora.dictionary import Dictionary
from gensim.models.tfidfmodel import TfidfModel

import os
import glob

import itertools 

# Building a Counter with bag-of-words

A continuación vamos a proceder nuestro primer contador de palabras, para ello vamos hacer uso de artículos de wikipedia.

In [2]:
#Cargamos los datos 
file = open('wiki_text_debugging.txt', mode = 'r')
info = file.read()

In [3]:
#Tokenizamos nuestro artículo
tokens = word_tokenize(info)

#Convertimos todos y cada uno de nuestros tokens a miniscula
lower_tokens = [t.lower() for t in tokens]

#Nos creamos nuestro objeto tipo Counter
bow_simple = Counter(lower_tokens)

#Mostramos los 10 tokens más comunes 
print(bow_simple.most_common(10))

[(',', 151), ('the', 150), ('.', 89), ('of', 81), ("''", 68), ('to', 63), ('a', 60), ('in', 44), ('and', 41), ('debugging', 40)]


# Text preprocessing practice

A la hora de trabajar con técnicas de procesado de lenguaje natural es muy conveniente realizar un pre-procesamiento de datos que nos permitirá tener nuestros datos de una forma adecuada, entre las técnicas más comunes se encuentran:

* Pasar todo a minúsculas.

* Eliminar signos de puntuación etc.

* Eliminar palabras sin significado (stopwords).

* Lematizar

In [4]:
#Tokenizamos
tokens = word_tokenize(info)

#Convertimos todos y cada uno de los tokens a minúscula
lower_tokens = [t.lower() for t in tokens]

#Nos quedamos con aquellos tokens alfabéticos
alpha_only = [t for t in lower_tokens if t.isalpha()]

#Eliminamos las palabras sin ningún tipo de significado
no_stops = [t for t in alpha_only if t not in stopwords.words('english')]

#Lematizamos
wordnet_lemmatizer = WordNetLemmatizer()
lemmatized = [wordnet_lemmatizer.lemmatize(t) for t in no_stops]

#Nos creamos el Counter
bow = Counter(lemmatized)

#Mostramos los 10 tokens más comunes 
print(bow.most_common(10))

[('debugging', 40), ('system', 25), ('software', 16), ('bug', 16), ('problem', 15), ('tool', 15), ('computer', 14), ('process', 13), ('term', 13), ('used', 12)]


# Creating and querying a corpus with gensim

Gensim se trata de una herramienta open source muy potente que nos facilita y permite realizar una gran cantidad de operaciones con texto de una forma sencilla y en pocas líneas de código.

In [5]:
#En primer lugar nos vamos a crear una función que nos permita aplicar todo el preprocesamiento visto
def preprocesa(texto):
    #Tokenizamos
    tokens = word_tokenize(texto)
    #Convertimos a minúscula
    lower_tokens = [t.lower() for t in tokens]
    #Eliminamos signos de puntuación etc
    alpha_only = [t for t in lower_tokens if t.isalpha()]
    #Eliminamos las palabras sin significado
    no_stops = [t for t in alpha_only if t not in stopwords.words('english')]
    #Lematizamos
    
    wordnet_lemmatizer = WordNetLemmatizer()
    lemmatized = [wordnet_lemmatizer.lemmatize(t) for t in no_stops]
    return(lemmatized)

In [6]:
#Obtenemos una lista con los ficheros txt
os.chdir(os.getcwd())
lista_ficheros = glob.glob("*.txt")

#Obtenemos una lista de listas donde cada lista de la lista principal será nuestro texto preprocesado
articles = []
for fichero in lista_ficheros:
    file = open(fichero, mode = 'r')
    info = file.read()
    articles.append(preprocesa(info))

In [7]:
#Nos creamos nuestro diccionario
dictionary = Dictionary(articles)

#Seleccionamos el id de la palabra computer
computer_id = dictionary.token2id.get('computer')

#Nos creamos nuestro corpus
corpus = [dictionary.doc2bow(article) for article in articles]

#Del t artículo mostramos sus 10 primeras palabras y su frencuencia de aparición
print(corpus[4][:10])

[(0, 1), (1, 1), (8, 3), (10, 3), (13, 10), (19, 2), (22, 1), (24, 2), (27, 1), (32, 3)]


Esta lista de tuplas indican el id del token y su frecuencia de aparición. Si queremos ver el token podemos hacer uso de **print(dictionary.get(token_id))**

# Gensim bag-of-words

A continuación vamos hacer uso de nuestro corpus y diccionario para ver los términos más comunes por documento y entre documentos.

In [8]:
#Obtenemos la información de nuestro cuarto documento
doc = corpus[4]

#Ordenamos por frecuencia de aparición los tokens de dicho documento
bow_doc = sorted(doc, key = lambda w: w[1], reverse = True)

#Obtenemos las 5 topics más comunes de nuestro documento
for word_id, word_count in bow_doc[:5]:
    print(dictionary.get(word_id), word_count)

software 160
computer 56
application 36
system 33
user 23


In [9]:
#Nos creamos un diccionario que contendrá la frecuencia de aparación de cada uno de nuestros tokens a lo largo de 
#los 12 documentos
total_word_count = defaultdict(int)
for word_id, word_count in itertools.chain.from_iterable(corpus):
    total_word_count[word_id] += word_count

In [10]:
#Ordenamos por frecuencia de aparición
sorted_word_count = sorted(total_word_count.items(), key = lambda w: w[1], reverse = True)

#Mostramos las 5 palabras más comunes 
for word_id, word_count in sorted_word_count[:5]:
    print(dictionary.get(word_id), word_count)

computer 749
software 450
program 340
cite 322
language 320


# Tf-idf with Wikipedia

La tf-idf se trata de una formula que nos permite ponderar la importancia de una palabra en corpus de documentos. Es decir, si estamos intentando extraer los topics de un conjunto de textos que hablan sobre baloncesto, es posible que las palabras como : canasta, baloncesto se repitan mucho pero realmente no están aportando ningún tipo de valor.

In [13]:
#Aplicamos la tf-idf
tfidf = TfidfModel(corpus)

#Calculamos la tfidf para el documento 5
tfidf_weights = tfidf[corpus[4]]

#Mostramos los 5 primeros pesos
print(tfidf_weights[:5])

[(0, 0.013505646340859068), (1, 0.0028126309499257561), (8, 0.01876503906999049), (10, 0.0084378928497772683), (19, 0.027011292681718136)]


In [14]:
#Ordenamos los pesos
sorted_tfidf_weights = sorted(tfidf_weights, key = lambda w: w[1], reverse = True)

#Vemos el top5 
for term_id, weight in sorted_tfidf_weights[:5]:
    print(dictionary[term_id], weight)

patent 0.299404430946
apis 0.230004355177
license 0.165846262831
application 0.159768085234
latter 0.153336236785
